# NYC Data Cleaning

* **DuckDB** is the safer default when you might hit very large data on very little memory. 
* **Polars** is fantastic for fast, expressive DataFrame work once the working set fits comfortably in RAM (or after you’ve trimmed it down).

### If you’re memory-constrained or the data is huge

* **Pick: DuckDB.**
* Why: disk-backed query engine, external (out-of-core) joins/groupbys, automatic spilling, great at reading many CSV/Parquet files and doing big merges without blowing RAM.
* Bonus: SQL is stable/portable, easy to hand off or productionize.

### If you want the fastest in-memory analytics in Python

* **Pick: Polars.**
* Why: Rust engine, lazy execution, predicate pushdown, superb groupbys/window ops, very fast on a single machine.
* Caveat: while Polars has streaming, not every operation is fully out-of-core; truly massive many-to-many joins still prefer DuckDB.

### A simple rule of thumb

* **Joins-first, massive ETL, union lots of files, ad-hoc SQL → DuckDB.**
* **Feature engineering, tidy transforms, complex expressions, post-aggregation work → Polars.**

### A solid “future-proof” workflow

1. **Stage & shrink with DuckDB** (read many raw files, filter, project, big joins, aggregate → write Parquet).
2. **Analyze with Polars** on the smaller Parquet outputs (fast iteration, modeling features, plotting).

* If you want one tool to start with for “extremely large on very low memory,” choose **DuckDB**. Add **Polars** to your toolkit for everything after the heavy lifting.
* If you dont want to use any of the above, in pandas limit the columns, remove any sorting operations, split the data month by month into parque files and merge one by one.


## Install Libs

In [23]:
import sys, platform
# print(sys.executable)
print(sys.version)
print(platform.platform())

3.11.9 (main, Oct  2 2025, 21:06:56) [Clang 16.0.0 (clang-1600.0.26.6)]
macOS-15.2-arm64-arm-64bit


In [2]:
%pip install --quiet pandas
%pip install --quiet numpy
%pip install --quiet duckdb
%pip install --quiet polars
%pip install --quiet pyarrow
# (the %pip magic binds to the current kernel’s interpreter)

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Import Libs

In [3]:
import pandas as pd
import numpy as np
import duckdb
import pyarrow
import polars as pl
from pathlib import Path

## Import Data

In [4]:
# For csv use
# pl.scan_csv("data.csv", has_header=True)
df_trips = pl.scan_parquet("datasets/yellow_tripdata_2024-01.parquet")
df_zones = pl.scan_parquet("datasets/taxi_zone_lookup.parquet")
df_weather = pl.scan_parquet('datasets/weather_data.parquet')
display(df_trips.head(2).collect())
display(df_zones.head(2).collect())
display(df_weather.head(2).collect())

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
i32,datetime[ns],datetime[ns],i64,f64,i64,str,i32,i32,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,"""N""",186,79,2,17.7,1.0,0.5,0.0,0.0,1.0,22.7,2.5,0.0
1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.8,1,"""N""",140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.0


LocationID,Borough,Zone,service_zone
i64,str,str,str
1,"""EWR""","""Newark Airport""","""EWR"""
2,"""Queens""","""Jamaica Bay""","""Boro Zone"""


year,month,day,hour,temp,temp_source,rhum,rhum_source,prcp,prcp_source,snwd,snwd_source,wdir,wdir_source,wspd,wspd_source,wpgt,wpgt_source,pres,pres_source,tsun,tsun_source,cldc,cldc_source,coco,coco_source
i64,i64,i64,i64,f64,str,i64,str,f64,str,i64,str,i64,str,f64,str,f64,str,f64,str,i64,str,i64,str,i64,str
2024,1,1,0,7.0,"""dwd_hourly""",79,"""dwd_hourly""",0.0,"""dwd_hourly""",0,"""dwd_poi""",210,"""dwd_hourly""",22.3,"""dwd_hourly""",32.0,"""dwd_poi""",1006.4,"""dwd_hourly""",0,"""dwd_poi""",8,"""dwd_hourly""",8,"""dwd_hourly"""
2024,1,1,1,6.9,"""dwd_hourly""",78,"""dwd_hourly""",0.0,"""dwd_hourly""",0,"""dwd_poi""",200,"""dwd_hourly""",20.5,"""dwd_hourly""",33.0,"""dwd_poi""",1006.6,"""dwd_hourly""",0,"""dwd_poi""",8,"""dwd_hourly""",7,"""dwd_poi"""


In [5]:
df_trips = pd.DataFrame({})


### DuckDB

In [6]:
def executeWithDuckdb():
    con = duckdb.connect()

    con.execute("""
    CREATE OR REPLACE TABLE trips_hour_zone AS
    WITH raw_trips AS (
      SELECT
        *,
        EXTRACT(EPOCH FROM (CAST(tpep_dropoff_datetime AS TIMESTAMP) - CAST(tpep_pickup_datetime AS TIMESTAMP))) / 60.0 AS trip_minutes,
        EXTRACT(EPOCH FROM (CAST(tpep_dropoff_datetime AS TIMESTAMP) - CAST(tpep_pickup_datetime AS TIMESTAMP))) AS trip_seconds
      FROM read_parquet('datasets/yellow_tripdata_2024-01.parquet')
      WHERE tpep_pickup_datetime IS NOT NULL
        AND tpep_dropoff_datetime IS NOT NULL
    )
    SELECT
      date_trunc('hour', CAST(tpep_pickup_datetime AS TIMESTAMP)) AS hour_local,
      CAST(PULocationID AS INTEGER) AS PULocationID,
      COUNT(*) AS trips,
      AVG(fare_amount) AS avg_fare,
      AVG(tip_amount) AS avg_tip,
      AVG(total_amount) AS avg_total,
      AVG(trip_distance) AS avg_distance,
      COALESCE(SUM(tip_amount) / NULLIF(SUM(total_amount), 0), 0) AS tip_pct,
      AVG(trip_minutes) AS avg_trip_minutes,
      SUM(total_amount) AS revenue_per_hour,
      SUM(trip_distance) AS total_distance_miles,
      AVG(
        CASE
          WHEN trip_seconds > 0
          THEN (trip_distance * 3600.0) / trip_seconds
        END
      ) AS avg_speed_mph
    FROM raw_trips
    WHERE PULocationID IS NOT NULL
    GROUP BY 1,2;
    """)

    con.execute("""
    CREATE OR REPLACE TABLE taxi_zone AS
    SELECT CAST(LocationID AS INTEGER) AS PULocationID, Borough, Zone, service_zone
    FROM read_parquet('datasets/taxi_zone_lookup.parquet');
    """)

    con.execute("""
    CREATE OR REPLACE TABLE weather AS
    SELECT
      make_timestamp(CAST(year AS INTEGER), CAST(month AS INTEGER),
                     CAST(day AS INTEGER), CAST(hour AS INTEGER), 0, 0) AS hour_local,
      CAST(temp AS DOUBLE) AS temp_c,
      CAST(rhum AS DOUBLE) AS humidity,
      CAST(prcp AS DOUBLE) AS precip_mm,
      CAST(wdir AS DOUBLE) AS wind_direction,
      CAST(wspd AS DOUBLE) AS wind_speed_kmh,
      CAST(pres AS DOUBLE) AS pressure_hpa,
      coco AS weather_condition
    FROM read_parquet('datasets/weather_data.parquet')
    WHERE year IS NOT NULL AND month IS NOT NULL AND day IS NOT NULL AND hour IS NOT NULL;
    """)

    df_trips = con.execute("""
    SELECT
      t.hour_local,
      t.PULocationID,
      z.Borough,
      z.Zone,
      z.service_zone,
      t.trips,
      t.avg_fare,
      t.avg_tip,
      t.avg_total,
      t.avg_distance,
      t.tip_pct,
      t.avg_trip_minutes,
      t.revenue_per_hour,
      t.total_distance_miles,
      t.avg_speed_mph,
      w.temp_c,
      w.humidity,
      w.precip_mm,
      w.wind_direction,
      w.wind_speed_kmh,
      w.pressure_hpa,
      w.weather_condition,
      date_trunc('day', t.hour_local) AS trip_date,
      CAST(EXTRACT(HOUR FROM t.hour_local) AS INTEGER) AS hour_of_day,
      STRFTIME(t.hour_local, '%A') AS day_of_week,
      t.avg_speed_mph * 1.60934 AS avg_speed_kmh,
      t.total_distance_miles * 1.60934 AS total_distance_km,
      t.tip_pct * 100 AS tip_pct_percent,
      CASE WHEN t.trips > 0 THEN t.revenue_per_hour / t.trips ELSE NULL END AS avg_revenue_per_trip
    FROM trips_hour_zone t
    LEFT JOIN weather w USING (hour_local)
    LEFT JOIN taxi_zone z USING (PULocationID)
    ORDER BY t.hour_local, t.PULocationID;
    """).df()

    column_order = [
        "hour_local",
        "PULocationID",
        "Borough",
        "Zone",
        "service_zone",
        "trips",
        "avg_fare",
        "avg_tip",
        "avg_total",
        "avg_distance",
        "tip_pct",
        "avg_trip_minutes",
        "revenue_per_hour",
        "total_distance_miles",
        "avg_speed_mph",
        "temp_c",
        "humidity",
        "precip_mm",
        "wind_direction",
        "wind_speed_kmh",
        "pressure_hpa",
        "weather_condition",
        "trip_date",
        "hour_of_day",
        "day_of_week",
        "avg_speed_kmh",
        "total_distance_km",
        "tip_pct_percent",
        "avg_revenue_per_trip",
    ]
    df_trips = df_trips[column_order]

    print("Executed by DuckDB")
    return df_trips


### Polars (Alternative to DuckDB)

In [7]:
def executeWithPolars():
    trip_minutes_expr = (
        pl.col("tpep_dropoff_datetime") - pl.col("tpep_pickup_datetime")
    ).dt.total_minutes()

    trips = (
        pl.scan_parquet("datasets/yellow_tripdata_2024-01.parquet")
        .with_columns([
            pl.col("tpep_pickup_datetime")
              .dt.truncate("1h")
              .dt.cast_time_unit("us")             # <- unify to μs
              .alias("hour_local"),
            pl.col("PULocationID").cast(pl.Int32),
            pl.col("tpep_dropoff_datetime"),
        ])
        .with_columns([
            trip_minutes_expr.alias("trip_minutes"),
            pl.when(trip_minutes_expr > 0)
              .then(pl.col("trip_distance") / (trip_minutes_expr / 60.0))
              .otherwise(None)
              .alias("speed_mph"),
        ])
        .group_by(["hour_local", "PULocationID"])
        .agg([
            pl.len().alias("trips"),
            pl.col("fare_amount").mean().alias("avg_fare"),
            pl.col("tip_amount").mean().alias("avg_tip"),
            pl.col("total_amount").mean().alias("avg_total"),
            pl.col("trip_distance").mean().alias("avg_distance"),
            pl.col("trip_minutes").mean().alias("avg_trip_minutes"),
            pl.col("total_amount").sum().alias("revenue_per_hour"),
            pl.col("trip_distance").sum().alias("total_distance_miles"),
            pl.col("speed_mph").mean().alias("avg_speed_mph"),
            pl.when(pl.sum("total_amount") != 0)
              .then(pl.sum("tip_amount") / pl.sum("total_amount"))
              .otherwise(0.0)
              .alias("tip_pct"),
        ])
    )

    zones = (
        pl.scan_parquet("datasets/taxi_zone_lookup.parquet")
        .select([pl.col("LocationID").cast(pl.Int32).alias("PULocationID"), "Borough", "Zone", "service_zone"])
    )

    weather = (
        pl.scan_parquet("datasets/weather_data.parquet")
        .with_columns([
            pl.datetime(
                pl.col("year").cast(pl.Int32),
                pl.col("month").cast(pl.Int32),
                pl.col("day").cast(pl.Int32),
                pl.col("hour").cast(pl.Int32),
                time_unit="us"                      # <- build as μs
            ).alias("hour_local")
        ])
        .select([
            "hour_local",
            pl.col("temp").alias("temp_c"),
            pl.col("rhum").alias("humidity"),
            pl.col("prcp").alias("precip_mm"),
            pl.col("wdir").alias("wind_direction"),
            pl.col("wspd").alias("wind_speed_kmh"),
            pl.col("pres").alias("pressure_hpa"),
            pl.col("coco").alias("weather_condition"),
        ])
    )

    df_trips = (
        trips.join(weather, on="hour_local", how="left")
             .join(zones, on="PULocationID", how="left")
             .with_columns([
                 pl.col("hour_local").dt.truncate("1d").alias("trip_date"),
                 pl.col("hour_local").dt.hour().cast(pl.Int32).alias("hour_of_day"),
                 pl.col("hour_local").dt.strftime('%A').alias("day_of_week"),
                 (pl.col("avg_speed_mph") * 1.60934).alias("avg_speed_kmh"),
                 (pl.col("total_distance_miles") * 1.60934).alias("total_distance_km"),
                 (pl.col("tip_pct") * 100).alias("tip_pct_percent"),
                 pl.when(pl.col("trips") > 0)
                   .then(pl.col("revenue_per_hour") / pl.col("trips"))
                   .otherwise(None)
                   .alias("avg_revenue_per_trip"),
             ])
             .sort(["hour_local", "PULocationID"])
             .collect()
             .to_pandas()
    )

    column_order = [
        "hour_local",
        "PULocationID",
        "Borough",
        "Zone",
        "service_zone",
        "trips",
        "avg_fare",
        "avg_tip",
        "avg_total",
        "avg_distance",
        "tip_pct",
        "avg_trip_minutes",
        "revenue_per_hour",
        "total_distance_miles",
        "avg_speed_mph",
        "temp_c",
        "humidity",
        "precip_mm",
        "wind_direction",
        "wind_speed_kmh",
        "pressure_hpa",
        "weather_condition",
        "trip_date",
        "hour_of_day",
        "day_of_week",
        "avg_speed_kmh",
        "total_distance_km",
        "tip_pct_percent",
        "avg_revenue_per_trip",
    ]
    df_trips = df_trips[column_order]

    print("Executed by Polars")
    return df_trips


### Pandas (Alternative to Polars)

In [8]:
def executeWithPandas():
    base = Path("datasets")
    trips = pd.read_parquet(base / "yellow_tripdata_2024-01.parquet", dtype_backend="pyarrow")

    trips["PULocationID"] = trips["PULocationID"].astype("Int64")
    trips["hour_local"] = trips["tpep_pickup_datetime"].dt.floor("h")  # -> timestamp[us][pyarrow]
    trips["trip_minutes"] = (trips["tpep_dropoff_datetime"] - trips["tpep_pickup_datetime"]).dt.total_seconds()/60
    trips["speed_mph"] = np.where(
        trips["trip_minutes"] > 0,
        trips["trip_distance"] / (trips["trip_minutes"] / 60),
        np.nan
    )

    agg = (
        trips.groupby(["hour_local","PULocationID"], dropna=False)
        .agg(
            trips=("VendorID","size"),
            avg_fare=("fare_amount","mean"),
            avg_tip=("tip_amount","mean"),
            avg_total=("total_amount","mean"),
            avg_distance=("trip_distance","mean"),
            tip_sum=("tip_amount","sum"),
            revenue_per_hour=("total_amount","sum"),
            avg_trip_minutes=("trip_minutes","mean"),
            total_distance_miles=("trip_distance","sum"),
            avg_speed_mph=("speed_mph","mean"),
        )
        .reset_index()
    )

    agg["tip_pct"] = np.where(
        agg["revenue_per_hour"].to_numpy() != 0,
        agg["tip_sum"] / agg["revenue_per_hour"],
        0.0,
    )
    agg = agg.drop(columns=["tip_sum"])

    agg["trip_date"] = agg["hour_local"].dt.floor('d')
    agg["hour_of_day"] = agg["hour_local"].dt.hour.astype('int32')
    agg["day_of_week"] = agg["hour_local"].dt.day_name()
    agg["avg_speed_kmh"] = agg["avg_speed_mph"] * 1.60934
    agg["total_distance_km"] = agg["total_distance_miles"] * 1.60934
    agg["tip_pct_percent"] = agg["tip_pct"] * 100
    agg["avg_revenue_per_trip"] = np.where(
        agg["trips"].to_numpy() > 0,
        agg["revenue_per_hour"] / agg["trips"],
        np.nan
    )

    zones = pd.read_parquet(base / "taxi_zone_lookup.parquet", dtype_backend="pyarrow").rename(columns={"LocationID":"PULocationID"})
    zones["PULocationID"] = pd.to_numeric(zones["PULocationID"], errors="coerce").astype("Int64")

    weather = pd.read_parquet(base / "weather_data.parquet", dtype_backend="pyarrow")
    for c in ["year","month","day","hour"]:
        weather[c] = pd.to_numeric(weather[c], errors="coerce").astype("Int64")

    weather["hour_local"] = pd.to_datetime(
        dict(year=weather["year"], month=weather["month"], day=weather["day"], hour=weather["hour"]),
        errors="coerce"
    )

    # align dtype with agg.hour_local (timestamp[us][pyarrow])
    weather["hour_local"] = weather["hour_local"].astype("timestamp[us][pyarrow]")

    weather = weather.rename(columns={
        "temp":"temp_c","rhum":"humidity","prcp":"precip_mm",
        "wdir":"wind_direction","wspd":"wind_speed_kmh",
        "pres":"pressure_hpa","coco":"weather_condition"
    })[
        ["hour_local","temp_c","humidity","precip_mm",
         "wind_direction","wind_speed_kmh","pressure_hpa","weather_condition"]
    ]

    df_trips = (
        agg.merge(weather, on="hour_local", how="left")
           .merge(zones, on="PULocationID", how="left")
           .sort_values(["hour_local","PULocationID"])
    )

    column_order = [
        "hour_local",
        "PULocationID",
        "Borough",
        "Zone",
        "service_zone",
        "trips",
        "avg_fare",
        "avg_tip",
        "avg_total",
        "avg_distance",
        "tip_pct",
        "avg_trip_minutes",
        "revenue_per_hour",
        "total_distance_miles",
        "avg_speed_mph",
        "temp_c",
        "humidity",
        "precip_mm",
        "wind_direction",
        "wind_speed_kmh",
        "pressure_hpa",
        "weather_condition",
        "trip_date",
        "hour_of_day",
        "day_of_week",
        "avg_speed_kmh",
        "total_distance_km",
        "tip_pct_percent",
        "avg_revenue_per_trip",
    ]
    df_trips = df_trips[column_order]

    print("Executed by Pandas")
    return df_trips


In [9]:
exec_method = "DUCKDB"
if exec_method == "DUCKDB":
    df_trips = executeWithDuckdb()
elif exec_method == "POLARS":
    df_trips = executeWithPolars()
else:
    df_trips = executeWithPandas()

display(df_trips.shape)
df_trips.head(3)


Executed by DuckDB


(77547, 29)

,hour_local,PULocationID,Borough,Zone,service_zone,trips,avg_fare,avg_tip,avg_total,avg_distance,...,wind_speed_kmh,pressure_hpa,weather_condition,trip_date,hour_of_day,day_of_week,avg_speed_kmh,total_distance_km,tip_pct_percent,avg_revenue_per_trip
0,2002-12-31 22:00:00,170,Manhattan,Murray Hill,Yellow Zone,2,0.0,0.0,0.00,0.63,...,NaN,NaN,<NA>,2002-12-31,22,Tuesday,10.082826,2.027768,0.0,0.00
1,2009-01-01 00:00:00,138,Queens,LaGuardia Airport,Airports,1,50.6,0.0,68.29,10.88,...,NaN,NaN,<NA>,2009-01-01,0,Thursday,21.506185,17.509619,0.0,68.29
2,2009-01-01 23:00:00,137,Manhattan,Kips Bay,Yellow Zone,1,4.4,0.0,9.40,0.46,...,NaN,NaN,<NA>,2009-01-01,23,Thursday,14.805928,0.740296,0.0,9.40


## Check Table Info

In [10]:
print("DUPLICATE COUNT------------------------------------------------------")
print(f"df_trips: {df_trips.duplicated().sum()}")
print(f"df_zones: {df_zones.collect().to_pandas().duplicated().sum()}")
print(f"df_weather: {df_weather.collect().to_pandas().duplicated().sum()}")

print("INFO------------------------------------------------------")
print(f"df_trips: {df_trips.info()}\n\n\n")
print(f"df_zones: {df_zones.collect().to_pandas().info()}\n\n\n")
print(f"df_weather: {df_weather.collect().to_pandas().info()}\n\n\n")

DUPLICATE COUNT------------------------------------------------------
df_trips: 0
df_zones: 0
df_weather: 0
INFO------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77547 entries, 0 to 77546
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hour_local            77547 non-null  datetime64[us]
 1   PULocationID          77547 non-null  int32         
 2   Borough               77547 non-null  object        
 3   Zone                  77547 non-null  object        
 4   service_zone          77547 non-null  object        
 5   trips                 77547 non-null  int64         
 6   avg_fare              77547 non-null  float64       
 7   avg_tip               77547 non-null  float64       
 8   avg_total             77547 non-null  float64       
 9   avg_distance          77547 non-null  float64       
 10  tip_pct               7

## Drop Duplicate Rows

In [11]:
df_trips.drop_duplicates(inplace=True)
df_trips.shape

(77547, 29)

## Check Missing Values

In [12]:
df_trips.isna().mean().round(4) * 100

hour_local              0.00
PULocationID            0.00
Borough                 0.00
Zone                    0.00
service_zone            0.00
trips                   0.00
avg_fare                0.00
avg_tip                 0.00
avg_total               0.00
avg_distance            0.00
tip_pct                 0.00
avg_trip_minutes        0.00
revenue_per_hour        0.00
total_distance_miles    0.00
avg_speed_mph           0.03
temp_c                  0.02
humidity                0.02
precip_mm               0.02
wind_direction          0.02
wind_speed_kmh          0.02
pressure_hpa            0.02
weather_condition       0.02
trip_date               0.00
hour_of_day             0.00
day_of_week             0.00
avg_speed_kmh           0.03
total_distance_km       0.00
tip_pct_percent         0.00
avg_revenue_per_trip    0.00
dtype: float64

In [13]:
# Quick sanity checks (optional)
# Confirm missingness isn’t clustered (i.e., not all at specific hours/locations):
# If it’s evenly spread, you’re good. If it clusters, consider an as-of join or light interpolation later.
na = df_trips['temp_c'].isna()
print(f"Mean: {na.mean()}\n\n")  # ~0.02
print(f"{df_trips.loc[na, 'hour_local'].dt.hour.value_counts(normalize=True).head()}\n\n")
print(df_trips.loc[na, 'PULocationID'].value_counts().head())


Mean: 0.00018053567513894798


hour_local
23    0.857143
22    0.071429
0     0.071429
Name: proportion, dtype: float64


PULocationID
138    2
170    1
137    1
237    1
68     1
Name: count, dtype: int64


* **Overall missing rate:** `0.0001805` ⇒ **0.018%** (6 rows out of ~33k).
* **When?** Mostly at **23:00** (late night), with a few at 22:00 and 00:00.
* **Where?** A handful of zones (PULocationID 138 twice; 170/137/237/68 once each).

That’s tiny and **localized at day boundaries**, which usually points to a *join-gap* (weather table missing the last hour of some days, or a 1-hour misalignment). What to do?

1. **Do nothing** → leave as NA. At 0.018%, this is totally acceptable for EDA, BI, and most tree models.
2. **Patch via nearest-hour join** (±1h tolerance) to catch those boundary hours.
3. **Drop those 6 rows** if you need fully numeric data right now and don’t want to impute.

If weather is missing `23:00` for those dates, that’s it. If not, check timezone alignment (ensure both trips and weather are in the **same local tz** before truncation).

So with only **6 rows** affected, you can safely **leave them as NA** or apply a simple **nearest-hour join** to clean them up.


In [14]:
# Keep tip_pct for Tableau metrics; any nulls indicate groups with zero recorded revenue.
df_trips["tip_pct"] = df_trips["tip_pct"].fillna(0.0)


## Handle Numeric Columns (Optional)

In [15]:
def convert_to_numeric(df_trips):
    numeric_fields = [
        'trips', 'avg_fare', 'avg_tip', 'avg_total', 'avg_distance',
        'avg_trip_minutes', 'revenue_per_hour', 'total_distance_miles',
        'total_distance_km', 'avg_speed_mph', 'avg_speed_kmh', 'temp_c',
        'humidity', 'precip_mm', 'wind_direction', 'wind_speed_kmh',
        'pressure_hpa', 'tip_pct', 'tip_pct_percent', 'avg_revenue_per_trip'
    ]
    existing = [c for c in numeric_fields if c in df_trips.columns]
    if not existing:
        return df_trips

    cleaned = df_trips[existing].copy()
    cleaned = cleaned.apply(
        lambda col: (
            col.astype('string')
               .str.replace('—', '-', regex=False)
               .str.replace('–', '-', regex=False)
               .str.replace('\u00A0', ' ', regex=False)
               .str.replace(',', '', regex=False)
               .str.replace(r'\$', '', regex=True)
               .str.strip()
        ) if col.dtype == 'object' else col
    )

    for col in existing:
        df_trips[col] = pd.to_numeric(cleaned[col], errors='coerce')

    df_trips[existing] = df_trips[existing].convert_dtypes()
    return df_trips

df_trips.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77547 entries, 0 to 77546
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hour_local            77547 non-null  datetime64[us]
 1   PULocationID          77547 non-null  int32         
 2   Borough               77547 non-null  object        
 3   Zone                  77547 non-null  object        
 4   service_zone          77547 non-null  object        
 5   trips                 77547 non-null  int64         
 6   avg_fare              77547 non-null  float64       
 7   avg_tip               77547 non-null  float64       
 8   avg_total             77547 non-null  float64       
 9   avg_distance          77547 non-null  float64       
 10  tip_pct               77547 non-null  float64       
 11  avg_trip_minutes      77547 non-null  float64       
 12  revenue_per_hour      77547 non-null  float64       
 13  total_distance_m

In [16]:
df_trips.select_dtypes(include=np.number).columns.tolist()

['PULocationID',
 'trips',
 'avg_fare',
 'avg_tip',
 'avg_total',
 'avg_distance',
 'tip_pct',
 'avg_trip_minutes',
 'revenue_per_hour',
 'total_distance_miles',
 'avg_speed_mph',
 'temp_c',
 'humidity',
 'precip_mm',
 'wind_direction',
 'wind_speed_kmh',
 'pressure_hpa',
 'weather_condition',
 'hour_of_day',
 'avg_speed_kmh',
 'total_distance_km',
 'tip_pct_percent',
 'avg_revenue_per_trip']

## Handle Date Time Columns (Optional)

In [17]:
def convert_to_datetime(df_trips):
    date_fields = [col for col in ['hour_local', 'trip_date'] if col in df_trips.columns]
    for col in date_fields:
        df_trips[col] = pd.to_datetime(df_trips[col], errors='coerce')
    return df_trips

df_trips.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77547 entries, 0 to 77546
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   hour_local            77547 non-null  datetime64[us]
 1   PULocationID          77547 non-null  int32         
 2   Borough               77547 non-null  object        
 3   Zone                  77547 non-null  object        
 4   service_zone          77547 non-null  object        
 5   trips                 77547 non-null  int64         
 6   avg_fare              77547 non-null  float64       
 7   avg_tip               77547 non-null  float64       
 8   avg_total             77547 non-null  float64       
 9   avg_distance          77547 non-null  float64       
 10  tip_pct               77547 non-null  float64       
 11  avg_trip_minutes      77547 non-null  float64       
 12  revenue_per_hour      77547 non-null  float64       
 13  total_distance_m

In [18]:
df_trips.select_dtypes(include=['datetime', 'datetimetz']).columns.tolist()

['hour_local', 'trip_date']

## Handle String Columns (Optional)

In [19]:
df_trips.head()

,hour_local,PULocationID,Borough,Zone,service_zone,trips,avg_fare,avg_tip,avg_total,avg_distance,...,wind_speed_kmh,pressure_hpa,weather_condition,trip_date,hour_of_day,day_of_week,avg_speed_kmh,total_distance_km,tip_pct_percent,avg_revenue_per_trip
0,2002-12-31 22:00:00,170,Manhattan,Murray Hill,Yellow Zone,2,0.0,0.00,0.00,0.63,...,NaN,NaN,<NA>,2002-12-31,22,Tuesday,10.082826,2.027768,0.000000,0.00
1,2009-01-01 00:00:00,138,Queens,LaGuardia Airport,Airports,1,50.6,0.00,68.29,10.88,...,NaN,NaN,<NA>,2009-01-01,0,Thursday,21.506185,17.509619,0.000000,68.29
2,2009-01-01 23:00:00,137,Manhattan,Kips Bay,Yellow Zone,1,4.4,0.00,9.40,0.46,...,NaN,NaN,<NA>,2009-01-01,23,Thursday,14.805928,0.740296,0.000000,9.40
3,2009-01-01 23:00:00,237,Manhattan,Upper East Side South,Yellow Zone,1,45.0,0.00,50.00,10.99,...,NaN,NaN,<NA>,2009-01-01,23,Thursday,34.232219,17.686647,0.000000,50.00
4,2023-12-31 23:00:00,68,Manhattan,East Chelsea,Yellow Zone,1,10.7,3.14,18.84,1.44,...,NaN,NaN,<NA>,2023-12-31,23,Sunday,14.409013,2.317450,16.666667,18.84


In [20]:
c = df_trips.select_dtypes(include=['object']).columns

def norm(c):
    return (
    c.astype("string")               # keeps NA as <NA>, not "nan"
     .str.replace("—", "-", regex=False)
     .str.replace("–", "-", regex=False)
     .str.replace("\u00A0", " ", regex=False)
     .str.replace(r"\\N", "N", regex=True)
     .str.strip()
)

def convert_to_object(df_trips):
    df_trips[c] = df_trips[c].apply(norm)
    df_trips.info()

In [21]:
display(df_trips.select_dtypes(include=['object']).columns.tolist())

['Borough', 'Zone', 'service_zone', 'day_of_week']

## Export Data

In [22]:
# To ensure all methods give same size output
output_file_type = "CSV"
if output_file_type == "CSV":
    df_trips.to_csv(
        "datasets/trips_complete.csv",
        index=False,
        na_rep="",
        float_format="%.6f",
        date_format="%Y-%m-%d %H:%M:%S",
        lineterminator="\n"
    )
else:
    # requires: pip install pyarrow
    df_trips.to_parquet(
        "datasets/trips_complete.parquet",
        engine="pyarrow",
        index=False,
        compression="zstd",          # strong + fast reads
        compression_level=12         # ~near-max without hurting read speed. Range: 1 to 22
    )

print("Done")

Done
